### Обработка данных

План:
- Будем работать с репликами менеджеров (конечно, в некоторых случаях, ответы клиента могут прояснить ситуацию, например когда расшифровка непонятна, но, в данной ситуации мб получится без этого обойтись)

- Для начала добавлю пунктуацию и прописные буквы (многие (все?) методы рассчитаны на использовании на отредактированном тексте, поэтому, хотя качество моделей автоматического редактирования оставляет желать лучшего это предпочтительнее, чем нулевой результат)

- Затем буду пользоваться стандартными средствами Yargy-parser'a (спасибо за подсказку в тексте вакансии!), дальнейшие комментарии буду оставлять по ходу дела 

-

In [1]:
import pandas as pd

data = pd.read_csv('test_data.csv')
data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [2]:
mng_data = data[data['role']=='manager'].drop('role', axis=1).reset_index(drop=True)
mng_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  201 non-null    int64 
 1   line_n  201 non-null    int64 
 2   text    201 non-null    object
dtypes: int64(2), object(1)
memory usage: 4.8+ KB


In [3]:
import torch

model, example_texts, languages, punct, apply_te = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                                                  model='silero_te')

Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


In [4]:
def enhance(input_text):
  # по какой то причине если добавить прописные буквы в текст все ломается
  #думаю нет никакой проблемы в том чтобы их убрать, поскольку они обозначают
  #лишь начало реплики, они обязательно восстановятся
  input_text = input_text.lower()
  letters = 'йцукенгшщзхъфывапролджэёячсмитьбюЙЦУКЕНГШЩЗХЪФЫВАПРОЛДЖЭЁЯЧСМИТЬБЮ%'
  if (len(input_text) >= 1) and (input_text[0] not in letters):
    if (len(input_text) >= 2) and (input_text[1] not in letters):
      if (len(input_text)>=3) and (input_text[2] not in letters):
        start = 0
      else:
        start = 2
    else:
      start = 1
  else:
    start = 0

  result = apply_te(input_text[start:], lan='ru')
  if start > 0:
    if len(result) == 1:
      result = result.lower()
    elif len(result) > 1:
      result = result[0].lower() + result[1:]

  result = input_text[:start] + result
  return result

In [5]:
mng_data['enhanced_text'] = mng_data['text'].apply(enhance)

In [6]:
!wget https://mydata.biz/storage/download/ebcdfe6fb2d546398010e0d6564a79bb/%D0%91%D0%B0%D0%B7%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B8%D0%BC%D0%B5%D0%BD%20%D0%B8%20%D1%84%D0%B0%D0%BC%D0%B8%D0%BB%D0%B8%D0%B9%20%D0%B2%20%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%82%D0%B5%20CSV.zip
!unzip 'База данных имен и фамилий в формате CSV.zip'

--2022-09-04 20:23:22--  https://mydata.biz/storage/download/ebcdfe6fb2d546398010e0d6564a79bb/%D0%91%D0%B0%D0%B7%D0%B0%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85%20%D0%B8%D0%BC%D0%B5%D0%BD%20%D0%B8%20%D1%84%D0%B0%D0%BC%D0%B8%D0%BB%D0%B8%D0%B9%20%D0%B2%20%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%82%D0%B5%20CSV.zip
Resolving mydata.biz (mydata.biz)... 217.174.101.25
Connecting to mydata.biz (mydata.biz)|217.174.101.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4665977 (4.4M) [application/zip]
Saving to: ‘База данных имен и фамилий в формате CSV.zip.1’

База данных имен и  100%[===================>]   4.45M   223KB/s    in 21s     

2022-09-04 20:23:45 (215 KB/s) - ‘База данных имен и фамилий в формате CSV.zip.1’ saved [4665977/4665977]

Archive:  База данных имен и фамилий в формате CSV.zip
replace _readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: _readme.txt             
replace foreign_names.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: fo

In [7]:
rusnames = pd.read_csv('russian_names.csv', sep=';')

In [8]:
names = rusnames.loc[rusnames['PeoplesCount']>=150,'Name'].values

In [9]:
!pip install natasha

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import numpy as np
from yargy.tokenizer import MorphTokenizer
from yargy import Parser, rule, and_, not_, or_
from yargy.predicates import gram, is_capitalized, dictionary, eq
NAME = rule(
    not_(gram('INTJ')), gram('Name')#фикс
)
pars_name = Parser(NAME)
#здесь я полагаю, что человеческие имена с прописной буквы сетка, обученная на сколько ни будь больших данных могла научиться хорошо
def extract_name(text, pars_name):
  for occurrency in pars_name.findall(text):
    for token in occurrency.tokens:
      if token.value in names:
        return token.value
  return 'No name'

mng_data['name'] = mng_data['enhanced_text'].apply(lambda x: extract_name(x, pars_name))
mng_data['named_herself'] = mng_data['name'] != 'No name'

In [11]:
from yargy.pipelines import morph_pipeline, caseless_pipeline
goodbye = ['До встречи', 'До свидания', 'Всего хорошего', 'Всего доброго' 'Всего наилучшего', 'Хорошего дня', 'Хорошего вечера']
GOODBYE = rule(caseless_pipeline(goodbye))
allg = rule('всего', 'доброго')
finl = or_(GOODBYE, allg)
gb_parser = Parser(finl)

In [12]:
def extract_goodbye(text):
  x = list(gb_parser.findall(text))
  if x:
    res = ' '.join([' '.join([y.value for y in z.tokens]) for z in x])
    return res
  return 'No goodbye'

In [13]:
mng_data['goodbye'] = mng_data['enhanced_text'].apply(lambda x: extract_goodbye(x))

In [14]:
mng_data[mng_data['goodbye']!='No goodbye']

,dlg_id,line_n,text,enhanced_text,name,named_herself,goodbye
15,0,108,Всего хорошего до свидания,Всего хорошего до свидания.,No name,False,Всего хорошего до свидания
43,1,53,Угу да вижу я эту почту хорошо тогда исправлю ...,"Угу, Да вижу я эту почту хорошо тогда исправлю...",No name,False,всего хорошего
44,1,54,До свидания,До свидания.,No name,False,До свидания
106,3,51,Угу все хорошо да понедельника тогда всего доб...,Угу все хорошо да понедельника тогда всего доб...,No name,False,всего доброго
120,4,33,Во вторник все ну с вами да тогда до вторника ...,Во вторник все ну с вами да тогда до вторника ...,No name,False,до свидания
200,5,142,Ну до свидания хорошего вечера,Ну до свидания хорошего вечера?,No name,False,до свидания хорошего вечера


In [15]:
from yargy.predicates import caseless
GREETING = or_(rule(
    caseless('Добрый'), gram('NOUN').optional() #добрый хочется сделать normalized но доброго вам дня на мой взгляд несколько устарело + возникают накладки со словами прощания
), rule(caseless('Здравствуйте')))

gr_parser = Parser(GREETING)

def extract_greeting(text):
  x = list(gr_parser.findall(text))
  if x:
    res = ' '.join([' '.join([y.value for y in z.tokens]) for z in x])
    return res
  return 'No hello'

mng_data['greeting'] = mng_data['enhanced_text'].apply(extract_greeting)

In [16]:
mng_data[mng_data['greeting']!='No hello']

,dlg_id,line_n,text,enhanced_text,name,named_herself,goodbye,greeting
0,0,1,Алло здравствуйте,Алло здравствуйте!,No name,False,No goodbye,здравствуйте
16,1,1,Алло здравствуйте,Алло здравствуйте!,No name,False,No goodbye,здравствуйте
45,2,2,Алло здравствуйте,Алло здравствуйте!,No name,False,No goodbye,здравствуйте
85,3,1,Алло дмитрий добрый день,Алло Дмитрий добрый день.,No name,False,No goodbye,добрый день
86,3,2,Добрый меня максим зовут компания китобизнес у...,Добрый меня Максим зовут компания Китобизнес у...,Максим,True,No goodbye,Добрый


In [17]:
adj_noun = or_(rule(gram('NOUN'), gram('ADJF'), gram('NOUN')), 
rule(gram('ADJF'), gram('NOUN')), 
rule(gram('NOUN').repeatable(max=3)))

In [18]:
cname_2 = rule(gram('Name'), eq(','), adj_noun)
cname_1 = rule(caseless('компания'), adj_noun)
cparser = Parser(or_(cname_1, cname_2))                             
for value in mng_data['enhanced_text']:                              
  x = list(cparser.findall(value))
  if x:
    x = x[0]
    print([y.value for y in x.tokens])

['Ангелина', ',', 'компания', 'диджитал', 'бизнес']
['Ангелина', ',', 'компания', 'диджитал', 'бизнес']
['Ангелина', ',', 'компания', 'диджитал', 'бизнес']
['компания', 'Китобизнес']


In [19]:
list(cparser.findall('Антон, компания Федеральная Служба Безопасности. Алексей, группа Ласковый Май, Никита, Детский мир'))

[Match(
     tokens=[MorphToken(
          value='Антон',
          span=[0, 5),
          type='RU',
          forms=[Form('антон', Grams(NOUN,Name,anim,masc,nomn,sing))]
      ), Token(
          value=',',
          span=[5, 6),
          type='PUNCT'
      ), MorphToken(
          value='компания',
          span=[7, 15),
          type='RU',
          forms=[Form('компания', Grams(NOUN,femn,inan,nomn,sing))]
      ), MorphToken(
          value='Федеральная',
          span=[16, 27),
          type='RU',
          forms=[Form('федеральный', Grams(ADJF,femn,nomn,sing))]
      ), MorphToken(
          value='Служба',
          span=[28, 34),
          type='RU',
          forms=[Form('служба', Grams(NOUN,femn,inan,nomn,sing))]
      )],
     span=[0, 34)
 ), Match(
     tokens=[MorphToken(
          value='Алексей',
          span=[49, 56),
          type='RU',
          forms=[Form('алексей', Grams(NOUN,Name,anim,masc,nomn,sing))]
      ), Token(
          value=',',
          span

In [23]:
nparser = Parser(rule(gram('Name')))
def get_company_name(text):
  x = list(cparser.findall(text))
  if x:
    x = x[0]
    res = [y.value for y in x.tokens]
    res = ' '.join(res)
    if res:
      name = list(nparser.findall(res))
      if name:
        namesp = list([x.tokens[0].span for x in name if x.tokens[0].value in names][0])
        res = res[:namesp[0]] + res[namesp[1]:]
    res = ' '.join([x for x in res.split() if x!=','])
    return res 
  return 'No company name'

mng_data['company_name'] = mng_data['enhanced_text'].apply(get_company_name)  

In [24]:
mng_data[mng_data['company_name']!='No company name']

,dlg_id,line_n,text,enhanced_text,name,named_herself,goodbye,greeting,company_name
1,0,3,Меня зовут ангелина компания диджитал бизнес з...,"Меня зовут Ангелина, компания диджитал бизнес,...",Ангелина,True,No goodbye,No hello,компания диджитал бизнес
17,1,2,Меня зовут ангелина компания диджитал бизнес з...,"Меня зовут Ангелина, компания диджитал бизнес ...",Ангелина,True,No goodbye,No hello,компания диджитал бизнес
46,2,3,Меня зовут ангелина компания диджитал бизнес з...,"Меня зовут Ангелина, компания диджитал бизнес ...",Ангелина,True,No goodbye,No hello,компания диджитал бизнес
86,3,2,Добрый меня максим зовут компания китобизнес у...,Добрый меня Максим зовут компания Китобизнес у...,Максим,True,No goodbye,Добрый,компания Китобизнес


In [25]:
from solution import ParserConvs
parser = ParserConvs()
reses = []
for dlg_id in data['dlg_id'].unique():
  reses.append(parser.parse(data.loc[data['dlg_id']==dlg_id]))

Using cache found in /root/.cache/torch/hub/snakers4_silero-models_master


In [26]:
for i, res in enumerate(reses):
  print('Dialog {}'.format(i))
  for key, value in res.items():
    print(key, ':', value)

Dialog 0
greeting : ['Алло здравствуйте']
named_herself : ['Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается']
name : ['Ангелина']
company_name : ['компания диджитал бизнес']
goodbye : ['Всего хорошего до свидания']
Dialog 1
greeting : ['Алло здравствуйте']
named_herself : ['Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается']
name : ['Ангелина']
company_name : ['компания диджитал бизнес']
goodbye : ['всего хорошего', 'До свидания']
Dialog 2
greeting : ['Алло здравствуйте']
named_herself : ['Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там']
name : ['Ангелина']
company_name : ['компания диджитал бизнес']
goodbye : []
Dialog 3
greeting : ['Алло дмитрий добрый день', 'Добрый меня максим зовут компания китобизнес удобно говорить']
named_herself : ['До